<a href="https://colab.research.google.com/github/shanxar/SCIKIT/blob/main/sk_heart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **USING HEART FAILURE DATASET , PERFORM SUPERVISED BINARY CLASSIFICATION TASK **

**WE USE LOGISITC REGRESSION MODEL **

In [21]:
import kagglehub

path = kagglehub.dataset_download("oxcolaa/heart-failure-prediction-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/heart-failure-prediction-dataset


In [22]:
import os
print(os.listdir(path))

['heart.xls']


In [23]:
import pandas as pd

#STEP 1: LOAD DATA

df=pd.read_csv(os.path.join(path,"heart.xls"))
print(f"DATASET \n {df}\n")

#STEP2: SPLIT FEATURES  AND LABELS
print(f"DATASET COLUMNS   : {df.columns}\n")
X=df.drop("HeartDisease",axis=1)
Y=df['HeartDisease']
print(f"X FEATURES :\n {X}\n")
print(f"Y LABELS :\n {Y}\n")


DATASET 
      Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  \
0     40   M           ATA        140          289          0     Normal   
1     49   F           NAP        160          180          0     Normal   
2     37   M           ATA        130          283          0         ST   
3     48   F           ASY        138          214          0     Normal   
4     54   M           NAP        150          195          0     Normal   
..   ...  ..           ...        ...          ...        ...        ...   
913   45   M            TA        110          264          0     Normal   
914   68   M           ASY        144          193          1     Normal   
915   57   M           ASY        130          131          0     Normal   
916   57   F           ATA        130          236          0        LVH   
917   38   M           NAP        138          175          0     Normal   

     MaxHR ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0      172       

In [28]:
#STEP 3 : SPLIT X ,Y TRAIN AND TEST

from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=20)

print(f"X_train Shape : {X_train.shape}")
print(f"Y_train Shape : {Y_train.shape}")
print(X_train.columns)
print(Y_train.name)


X_train Shape : (642, 11)
Y_train Shape : (642,)
Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope'],
      dtype='object')
HeartDisease


In [43]:
#STEP 4 : GOTTA NORMALIZE THE NUMERICAL FEATURES AND ENCODE THE CATEGROICAL STRING FEATURES

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder

#split the numerical columns and string columns
#use X_train for this, cus we dont want to take label column to consideration
num_cols= X_train.select_dtypes(include="int64").columns.tolist()
cat_cols=X_train.select_dtypes(include="object").columns.tolist()
print(f"NUMERICAL COLUMNS : {num_cols}\nCATEGORICAL COLUMNS : {cat_cols}\n")

#USE COLUMN TRANFORMER TO APPLY ENCODING FOR CATEGORICAL COLS AND NORMMALIZING FOR NUM COLUMNS

col_trans=ColumnTransformer([
                            ("num",StandardScaler(),num_cols),
                            ("cat",OneHotEncoder(sparse_output=False),cat_cols)
])

X_train_preprocessed= col_trans.fit_transform(X_train)
X_test_preprocessed= col_trans.transform(X_test) #These return numpy ndarray

#STEP  5 : CONNVERT BACKK TO DATAFRAME
#Get col names of the numpy ndarray (X_train_preprocessed and x_test_preprocesed)
col_names=col_trans.get_feature_names_out()
print(col_names)

#Create Dataframe out of it

X_train_preprocessed=pd.DataFrame(X_train_preprocessed,columns=col_names)
print(X_train_preprocessed)

X_test_preprocessed=pd.DataFrame(X_test_preprocessed,columns=col_names)
print(X_test_preprocessed)

NUMERICAL COLUMNS : ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR']
CATEGORICAL COLUMNS : ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

['num__Age' 'num__RestingBP' 'num__Cholesterol' 'num__FastingBS'
 'num__MaxHR' 'cat__Sex_F' 'cat__Sex_M' 'cat__ChestPainType_ASY'
 'cat__ChestPainType_ATA' 'cat__ChestPainType_NAP' 'cat__ChestPainType_TA'
 'cat__RestingECG_LVH' 'cat__RestingECG_Normal' 'cat__RestingECG_ST'
 'cat__ExerciseAngina_N' 'cat__ExerciseAngina_Y' 'cat__ST_Slope_Down'
 'cat__ST_Slope_Flat' 'cat__ST_Slope_Up']
     num__Age  num__RestingBP  num__Cholesterol  num__FastingBS  num__MaxHR  \
0    1.606756        0.208655         -1.851913       -0.552158   -1.300036   
1    0.442134        0.900525          0.176358       -0.552158   -0.753868   
2   -0.193114        1.698837         -1.851913        1.811077   -0.597820   
3    0.865633       -0.163890          0.287496       -0.552158    0.338469   
4   -1.145986       -0.696098          0.843187  

In [48]:
#IF THE CATEGORICAL DATA WAS THERE WE USE LABLE ENCODING OR ONE HOT ENCODING
#SINCE WE HAVE 0 OR 1 BINARY WE SKIP THE STEP

#STEP5:TRAIN MODEL

#WE USE LOGISTIC REGRESSION MODEL HERE, WHICH PREDICTS PROBABIULITY OF HEART DISEASE AND USE SIGMOID FUNCTION INTERNALLY TO SQUASH PROBABILITY NUMBERS TO 0 OR 1

from sklearn.linear_model import LogisticRegression


log_reg=LogisticRegression(max_iter=10000) # CREATE OBJECT FOR MODEL

log_reg.fit(X_train_preprocessed,Y_train)

#STEP 6 : PREDICTION

y_pred=log_reg.predict(X_test_preprocessed)

print(y_pred)



[1 1 1 1 0 0 1 1 0 1 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 0 1 0 0 1 1 1 0 0 1 1 1
 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 1 1 1 1 1 0 0 1 0 0 0 1 1 1 1 0 0 1 0 0 1 0
 1 1 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 0 1 1 1
 0 0 0 1 1 1 1 0 0 0 0 0 1 1 0 1 0 0 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 1 1
 1 1 1 0 0 1 1 1 0 1 1 0 1 1 0 1 1 0 0 0 1 1 1 1 0 0 0 0 1 1 0 0 0 1 0 0 1
 1 1 0 0 1 1 0 0 1 0 0 1 1 1 0 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 1 1 1 1 1 1
 0 1 0 1 0 1 1 0 1 0 1 1 1 1 0 0 1 1 0 1 0 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0
 0 1 1 1 0 1 1 1 0 1 0 1 1 0 1 0 1]


In [51]:
#STEP 7 : EVALUATION PERFORMANCE

from sklearn.metrics import confusion_matrix,accuracy_score

accuracy=accuracy_score(Y_test,y_pred)
print(f"ACCURACY SCORE : {accuracy}")
print(f"CONFUSION MATRIX \n: {confusion_matrix(Y_test,y_pred)}")

ACCURACY SCORE : 0.8659420289855072
CONFUSION MATRIX 
: [[103  24]
 [ 13 136]]
